### Data processing steps

In [1]:
import os
import csv
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,Cropping2D
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Flatten, SpatialDropout2D , ELU
from keras.layers.core import Lambda
from keras.optimizers import Adam
from keras.utils import np_utils
from PIL import Image
import pickle

%matplotlib inline

Using TensorFlow backend.


---------

### 1. Read main File

In [8]:
info = pd.read_csv('driving_log.csv')
info.head()

,center,left,right,steering,throttle,brake,speed
0,IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
1,IMG/center_2016_12_01_13_30_48_404.jpg,IMG/left_2016_12_01_13_30_48_404.jpg,IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
2,IMG/center_2016_12_01_13_31_12_937.jpg,IMG/left_2016_12_01_13_31_12_937.jpg,IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011
3,IMG/center_2016_12_01_13_31_13_037.jpg,IMG/left_2016_12_01_13_31_13_037.jpg,IMG/right_2016_12_01_13_31_13_037.jpg,0.0,0.0,0.0,1.438419
4,IMG/center_2016_12_01_13_31_13_177.jpg,IMG/left_2016_12_01_13_31_13_177.jpg,IMG/right_2016_12_01_13_31_13_177.jpg,0.0,0.0,0.0,1.418236


In [9]:
info.shape

(8036, 7)

### 2. Seperate them to 3 different files

#### 2.1. Create center file

In [10]:
center_info = pd.DataFrame()
center_info['path'] = info['center']
center_info['steering'] = info['steering']
center_info.head()

,path,steering
0,IMG/center_2016_12_01_13_30_48_287.jpg,0.0
1,IMG/center_2016_12_01_13_30_48_404.jpg,0.0
2,IMG/center_2016_12_01_13_31_12_937.jpg,0.0
3,IMG/center_2016_12_01_13_31_13_037.jpg,0.0
4,IMG/center_2016_12_01_13_31_13_177.jpg,0.0


#### 2.2. Create right file

In [11]:
right_info = pd.DataFrame()
right_info['path'] = info['right']
right_info['steering'] = info['steering']
right_info.head()

,path,steering
0,IMG/right_2016_12_01_13_30_48_287.jpg,0.0
1,IMG/right_2016_12_01_13_30_48_404.jpg,0.0
2,IMG/right_2016_12_01_13_31_12_937.jpg,0.0
3,IMG/right_2016_12_01_13_31_13_037.jpg,0.0
4,IMG/right_2016_12_01_13_31_13_177.jpg,0.0


#### 2.3. Create left file

In [12]:
left_info = pd.DataFrame()
left_info['path'] = info['left']
left_info['steering'] = info['steering']
left_info.head()

,path,steering
0,IMG/left_2016_12_01_13_30_48_287.jpg,0.0
1,IMG/left_2016_12_01_13_30_48_404.jpg,0.0
2,IMG/left_2016_12_01_13_31_12_937.jpg,0.0
3,IMG/left_2016_12_01_13_31_13_037.jpg,0.0
4,IMG/left_2016_12_01_13_31_13_177.jpg,0.0


--------

### 3. Process right and left files

In [13]:
right_info['steering'] = right_info['steering'].apply(lambda x : x-0.2) 
right_info.head()

,path,steering
0,IMG/right_2016_12_01_13_30_48_287.jpg,-0.2
1,IMG/right_2016_12_01_13_30_48_404.jpg,-0.2
2,IMG/right_2016_12_01_13_31_12_937.jpg,-0.2
3,IMG/right_2016_12_01_13_31_13_037.jpg,-0.2
4,IMG/right_2016_12_01_13_31_13_177.jpg,-0.2


In [14]:
 left_info['steering'] = left_info['steering'].apply(lambda x : x+0.2) 
left_info.head()

,path,steering
0,IMG/left_2016_12_01_13_30_48_287.jpg,0.2
1,IMG/left_2016_12_01_13_30_48_404.jpg,0.2
2,IMG/left_2016_12_01_13_31_12_937.jpg,0.2
3,IMG/left_2016_12_01_13_31_13_037.jpg,0.2
4,IMG/left_2016_12_01_13_31_13_177.jpg,0.2


-------

### 4. Concatenet all 3 files to a single file

In [49]:
alog = pd.concat([center_info,right_info,left_info])

In [50]:
alog.shape

(24108, 2)

In [ ]:
alog.head()

------

In [51]:
alog.to_csv('alog.csv')

In [ ]:
r_alog = pd.read_csv('alog.csv')
r_alog.head()

------

### 5. Data Augmentation

In [53]:
import csv
from PIL import Image
import pandas as pd
import numpy as np

In [54]:
samples = alog[0:5]

In [67]:
for item in samples.iterrows():
    print(item[1][0])
    print(item[1][1])
    
    

IMG/center_2016_12_01_13_30_48_287.jpg
0.0
IMG/center_2016_12_01_13_30_48_404.jpg
0.0
IMG/center_2016_12_01_13_31_12_937.jpg
0.0
IMG/center_2016_12_01_13_31_13_037.jpg
0.0
IMG/center_2016_12_01_13_31_13_177.jpg
0.0


In [70]:
image_f_info = {'path':[],'steering':[]}
for item in alog.iterrows():
        
        path = item[1][0]
        if path[0] ==' ':
            path = path[1:]
            
        # flip the image
        #image = Image.open(path)
        #image_f = image.transpose(Image.FLIP_LEFT_RIGHT)
        # save flipped image
        #path_f = path.split('.')[0] + '_f.jpg'
        #image_f.save(path_f)
        
        image_f_info['path'].append(path_f)
        
        f_steering_angle = -1. * float(item[1][1])
        image_f_info['steering'].append(f_steering_angle)
    
flog = pd.DataFrame(image_f_info)

print('flip completed...')

flip completed...


In [71]:
print(flog.head(5))

                                       path  steering
0  IMG/center_2016_12_01_13_30_48_287_f.jpg      -0.0
1  IMG/center_2016_12_01_13_30_48_404_f.jpg      -0.0
2  IMG/center_2016_12_01_13_31_12_937_f.jpg      -0.0
3  IMG/center_2016_12_01_13_31_13_037_f.jpg      -0.0
4  IMG/center_2016_12_01_13_31_13_177_f.jpg      -0.0


-------

In [74]:
flog.to_csv('flog.csv')

In [75]:
r_flog = pd.read_csv('flog.csv')
r_flog.head()

,Unnamed: 0,path,steering
0,0,IMG/center_2016_12_01_13_30_48_287_f.jpg,-0.0
1,1,IMG/center_2016_12_01_13_30_48_404_f.jpg,-0.0
2,2,IMG/center_2016_12_01_13_31_12_937_f.jpg,-0.0
3,3,IMG/center_2016_12_01_13_31_13_037_f.jpg,-0.0
4,4,IMG/center_2016_12_01_13_31_13_177_f.jpg,-0.0


-------

#### Concate All filpped and unflipped file

In [77]:
log = pd.concat([alog,flog])

In [78]:
log.to_csv('log.csv')

In [79]:
r_log = pd.read_csv('log.csv')
r_log.head()

,Unnamed: 0,path,steering
0,0,IMG/center_2016_12_01_13_30_48_287.jpg,0.0
1,1,IMG/center_2016_12_01_13_30_48_404.jpg,0.0
2,2,IMG/center_2016_12_01_13_31_12_937.jpg,0.0
3,3,IMG/center_2016_12_01_13_31_13_037.jpg,0.0
4,4,IMG/center_2016_12_01_13_31_13_177.jpg,0.0


In [80]:
r_log.tail()

,Unnamed: 0,path,steering
48211,24103,IMG/left_2016_12_01_13_46_38_745_f.jpg,-0.2
48212,24104,IMG/left_2016_12_01_13_46_38_802_f.jpg,-0.2
48213,24105,IMG/left_2016_12_01_13_46_38_846_f.jpg,-0.2
48214,24106,IMG/left_2016_12_01_13_46_38_922_f.jpg,-0.2
48215,24107,IMG/left_2016_12_01_13_46_38_947_f.jpg,-0.2
